In [1]:
pip install numpy


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pandas


[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install -e ./electre_library

Obtaining file:///Users/mrezkys/Kampus/SPK/electre_library
  Preparing metadata (setup.py) ... done
  Attempting uninstall: electre_dss
    Found existing installation: electre_dss 0.1.0
    Uninstalling electre_dss-0.1.0:
      Successfully uninstalled electre_dss-0.1.0
  DEPRECATION: Legacy editable install of electre_dss==0.1.0 from file:///Users/mrezkys/Kampus/SPK/electre_library (setup.py develop) is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for electre_dss

[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.4.1_1/libexec/bin/python -m p

In [4]:
import numpy as np
import pandas as pd
from electre_dss import ElectreSolver

In [11]:
# 1. Initialize Data
decision_matrix = np.array([
    [86.33, 86.33, 87.60, 87.33], # A1
    [84.33,	85,	87, 89], # A2
    [85, 85.6, 86.6, 87.6], # A3
    [84.33, 83.6, 87.6, 87.33], # A4
    [85.33, 85.33, 86.33, 87], # A5
])
weights = np.array([0.2, 0.15, 0.30, 0.35])
criteria_types = ['benefit', 'benefit', 'benefit', 'benefit']
alternative_names = ['Saufan Ridho', 'Christian Hadi Chandra', 'Alya Dwi Pangesti', 'Dzakanov Inshoofi', 'Sucitasari Rahmadani']
criteria_names = ['C1(Komitment)', 'C2(Kepribadian)', 'C3(Cara Mengajar)', 'C4(Pemahaman Materi)']

# 2. Initialize and run the solver
print("\n--- Running Solver ---")
solver = ElectreSolver(
    decision_matrix=decision_matrix,
    weights=weights,
    criteria_types=criteria_types,
    alternative_names=alternative_names,
    criteria_names=criteria_names
)

solver.solve() # performs all calculations

# 3. Get and print the final ranking
ranking = solver.get_ranking()
print("\n------ FINAL RANKING ------")
for i, name in enumerate(ranking):
    print(f"{i+1}. {name}")


--- Running Solver ---

------ FINAL RANKING ------
1. Christian Hadi Chandra
2. Saufan Ridho
3. Dzakanov Inshoofi
4. Alya Dwi Pangesti
5. Sucitasari Rahmadani



--- Running Solver ---



------ FINAL RANKING ------
1. Christian Hadi Chandra
2. Saufan Ridho
3. Dzakanov Inshoofi
4. Alya Dwi Pangesti
5. Sucitasari Rahmadani


In [8]:
# 4. Display detailed intermediate results for verification
print("\n------ DETAILED STEP-BY-STEP RESULTS ------")
results = solver.get_intermediate_results()
alternative_names = solver.alternative_names # Get names from solver
criteria_names = solver.criteria_names     # Get names from solver

# Step 1: Normalization (Euclidean)
print("\nStep 1: Normalized Decision Matrix (R)")
display(pd.DataFrame(results['raw_normalized_matrix'], index=alternative_names, columns=criteria_names).round(3))

# Step 2: Weighted Normalization
print("\nStep 2: Weighted Normalized Matrix (V)")
display(pd.DataFrame(results['weighted_matrix'], index=alternative_names, columns=criteria_names).round(3))

# Step 3 & 4: Concordance/Discordance Sets & Indices
print("\nSteps 3 & 4: Concordance and Discordance Analysis")
# Ensure global max diff is available and formatted correctly
global_max_diff_val = results['discordance_global_max_diff']
print(f"Global Max Difference for Discordance Denominator: {global_max_diff_val:.2f}")

for k in range(solver.num_alternatives):
    for l in range(solver.num_alternatives):
        if k == l: continue
        k_name = alternative_names[k]
        l_name = alternative_names[l]
        pair = (k, l) # Use integer indices for dict keys

        print(f'\n--- Pair ({k_name} vs {l_name}) ---')
        
        # Concordance
        # Use .get(pair, []) for safety if a pair somehow wasn't calculated
        conc_set = results['concordance_sets'].get(pair, []) 
        conc_index = results['concordance_matrix'][k, l]
        # Format set output nicely
        conc_set_str = ', '.join(conc_set) if conc_set else 'None' 
        print(f"  Concordance Set C({k_name},{l_name}): {{ {conc_set_str} }}")
        print(f"  Concordance Index C_{k+1},{l+1}: {conc_index:.1f}") # Match report format (1 decimal usually)

        # Discordance
        disc_set = results['discordance_sets'].get(pair, [])
        # Get max diff within the set for this pair
        disc_max_diff = results['discordance_max_diffs'].get(pair, 0.0) 
        disc_index = results['discordance_matrix'][k, l]
        disc_set_str = ', '.join(disc_set) if disc_set else 'None'
        print(f"  Discordance Set D({k_name},{l_name}): {{ {disc_set_str} }}")
        print(f"  Max Difference in D({k_name},{l_name}): {disc_max_diff:.2f}")
        # Show the calculation clearly
        print(f"  Discordance Index d_{k+1},{l+1}: ({disc_max_diff:.2f} / {global_max_diff_val:.2f}) = {disc_index:.2f}")

# Step 5: Thresholds
print("\nStep 5: Threshold Calculation")
c_threshold = results['concordance_threshold']
d_threshold = results['discordance_threshold']
print(f"  Concordance Threshold (c) = Sum(C_kl) / (m*(m-1)) = {np.sum(results['concordance_matrix']):.1f} / {solver.num_alternatives * (solver.num_alternatives - 1)} = {c_threshold:.2f}")
print(f"  Discordance Threshold (d) = Sum(d_kl) / (m*(m-1)) = {np.sum(results['discordance_matrix']):.2f} / {solver.num_alternatives * (solver.num_alternatives - 1)} = {d_threshold:.4f}")

# Step 6: Dominance Matrices
print("\nStep 6: Dominance Matrices")
print(f"  Concordance Dominance Matrix (F): C_kl >= {c_threshold:.2f}")
display(pd.DataFrame(results['concordance_dominance_matrix'], index=alternative_names, columns=alternative_names))
print(f"  Discordance Dominance Matrix (G): d_kl <= {d_threshold:.4f}")
display(pd.DataFrame(results['discordance_dominance_matrix'], index=alternative_names, columns=alternative_names))

# Step 7: Aggregate Dominance Matrix
print("\nStep 7: Aggregate Dominance Matrix (E) = F * G")
display(pd.DataFrame(results['aggregate_dominance_matrix'], index=alternative_names, columns=alternative_names))

# Step 8: Final Ranking (derived from E)
print("\nStep 8: Final Ranking (interpreting E)")
ranking = solver.get_ranking() # Get ranking calculated by the solver
# Explanation based on the simple net score used in _determine_ranking
print("  (Ranking based on net dominance score: sum(row E) - sum(col E))") 
for i, name in enumerate(ranking):
    print(f"  {i+1}. {name}")


------ DETAILED STEP-BY-STEP RESULTS ------

Step 1: Normalized Decision Matrix (R)


C1(Komitment)  C2(Kepribadian)  C3(Cara Mengajar)  \
Saufan Ridho                    0.454            0.453              0.450   
Christian Hadi Chandra          0.443            0.446              0.447   
Alya Dwi Pangesti               0.447            0.449              0.445   
Dzakanov Inshoofi               0.443            0.439              0.450   
Sucitasari Rahmadani            0.449            0.448              0.444   

                        C4(Pemahaman Materi)  
Saufan Ridho                           0.446  
Christian Hadi Chandra                 0.454  
Alya Dwi Pangesti                      0.447  
Dzakanov Inshoofi                      0.446  
Sucitasari Rahmadani                   0.444


Step 2: Weighted Normalized Matrix (V)


C1(Komitment)  C2(Kepribadian)  C3(Cara Mengajar)  \
Saufan Ridho                    0.091            0.068              0.135   
Christian Hadi Chandra          0.089            0.067              0.134   
Alya Dwi Pangesti               0.089            0.067              0.134   
Dzakanov Inshoofi               0.089            0.066              0.135   
Sucitasari Rahmadani            0.090            0.067              0.133   

                        C4(Pemahaman Materi)  
Saufan Ridho                           0.156  
Christian Hadi Chandra                 0.159  
Alya Dwi Pangesti                      0.156  
Dzakanov Inshoofi                      0.156  
Sucitasari Rahmadani                   0.155


Steps 3 & 4: Concordance and Discordance Analysis
Global Max Difference for Discordance Denominator: 0.00

--- Pair (Saufan Ridho vs Christian Hadi Chandra) ---
  Concordance Set C(Saufan Ridho,Christian Hadi Chandra): { C1(Komitment), C2(Kepribadian), C3(Cara Mengajar) }
  Concordance Index C_1,2: 0.6
  Discordance Set D(Saufan Ridho,Christian Hadi Chandra): { C4(Pemahaman Materi) }
  Max Difference in D(Saufan Ridho,Christian Hadi Chandra): 0.00
  Discordance Index d_1,2: (0.00 / 0.00) = 0.83

--- Pair (Saufan Ridho vs Alya Dwi Pangesti) ---
  Concordance Set C(Saufan Ridho,Alya Dwi Pangesti): { C1(Komitment), C2(Kepribadian), C3(Cara Mengajar) }
  Concordance Index C_1,3: 0.6
  Discordance Set D(Saufan Ridho,Alya Dwi Pangesti): { C4(Pemahaman Materi) }
  Max Difference in D(Saufan Ridho,Alya Dwi Pangesti): 0.00
  Discordance Index d_1,3: (0.00 / 0.00) = 0.13

--- Pair (Saufan Ridho vs Dzakanov Inshoofi) ---
  Concordance Set C(Saufan Ridho,Dzakanov Inshoofi): { C1(Komitment), C2(Ke

...

  Discordance Dominance Matrix (G): d_kl <= 0.4313


Saufan Ridho  Christian Hadi Chandra  \
Saufan Ridho                       0                       0   
Christian Hadi Chandra             0                       0   
Alya Dwi Pangesti                  0                       0   
Dzakanov Inshoofi                  0                       0   
Sucitasari Rahmadani               0                       0   

                        Alya Dwi Pangesti  Dzakanov Inshoofi  \
Saufan Ridho                            1                  1   
Christian Hadi Chandra                  1                  1   
Alya Dwi Pangesti                       0                  0   
Dzakanov Inshoofi                       0                  0   
Sucitasari Rahmadani                    1                  0   

                        Sucitasari Rahmadani  
Saufan Ridho                               1  
Christian Hadi Chandra                     1  
Alya Dwi Pangesti                          1  
Dzakanov Inshoofi                          1  
Sucitasari Rahmadani                       0


Step 7: Aggregate Dominance Matrix (E) = F * G


Saufan Ridho  Christian Hadi Chandra  \
Saufan Ridho                       0                       0   
Christian Hadi Chandra             0                       0   
Alya Dwi Pangesti                  0                       0   
Dzakanov Inshoofi                  0                       0   
Sucitasari Rahmadani               0                       0   

                        Alya Dwi Pangesti  Dzakanov Inshoofi  \
Saufan Ridho                            1                  1   
Christian Hadi Chandra                  1                  1   
Alya Dwi Pangesti                       0                  0   
Dzakanov Inshoofi                       0                  0   
Sucitasari Rahmadani                    0                  0   

                        Sucitasari Rahmadani  
Saufan Ridho                               1  
Christian Hadi Chandra                     1  
Alya Dwi Pangesti                          1  
Dzakanov Inshoofi                          1  
Sucitasari Rahmadani                       0


Step 8: Final Ranking (interpreting E)
  (Ranking based on net dominance score: sum(row E) - sum(col E))
  1. Christian Hadi Chandra
  2. Saufan Ridho
  3. Dzakanov Inshoofi
  4. Alya Dwi Pangesti
  5. Sucitasari Rahmadani
